Prueba Modelo redes neuronales dato IPC (1191 - 2025)

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

2025-02-18 21:17:10.212156: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-18 21:17:10.391933: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-18 21:17:10.454894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739924230.540525   16842 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739924230.558755   16842 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 21:17:11.181312: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
data = "data/raw_Data_Econocast.csv"  # Asegúrate de que el nombre es correcto
df_ipc = pd.read_csv(data)
df_ipc.head()

,anio,mes,precio_promedio_venta_USD,promedio_valor_USD_anual,alimentos_bebidas_sin_alcohol,bebidas_alcoholicas_tabaco,prendas_de_vestir_calzado,alojamiento_agua_electricidad_gas_otroscomb,muebles_articulos_hogar,salud,...,comunicaciones,recreacion _cultura,educacion,restaurantes_hoteles,cuidado_personal,indice_ipc_general,importaciones_valor_usd,importaciones_volumen_kg,exportaciones_valor_usd,exportaciones_volumen_kg
0,1991,Enero,NaN,NaN,23.82,23.15,26.14,24.62,39.65,18.53,...,68.38,17.08,18.39,16.69,27.35,24.09,NaN,NaN,NaN,NaN
1,1991,Febrero,NaN,NaN,24.99,24.79,27.11,24.93,40.17,20.12,...,34.33,17.84,22.05,18.37,27.86,24.88,NaN,NaN,NaN,NaN
2,1991,Marzo,NaN,NaN,25.45,27.02,27.97,25.11,40.58,20.22,...,35.18,17.84,22.05,18.37,28.10,25.21,NaN,NaN,NaN,NaN
3,1991,Abril,NaN,NaN,25.89,26.97,28.81,26.25,42.23,20.52,...,70.55,18.17,22.05,18.37,28.49,26.04,NaN,NaN,NaN,NaN
4,1991,Mayo,NaN,NaN,26.60,27.33,29.34,26.39,42.46,20.97,...,36.76,19.23,22.05,19.45,28.47,26.34,NaN,NaN,NaN,NaN


In [3]:
meses_dict = {
    "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4, "Mayo": 5, "Junio": 6,
    "Julio": 7, "Agosto": 8, "Septiembre": 9, "Octubre": 10, "Noviembre": 11, "Diciembre": 12
}

In [4]:
df_ipc["mes"] = df_ipc["mes"].map(meses_dict)

In [5]:
# Convertir a formato de fecha
df_ipc["fecha"] = pd.to_datetime(df_ipc["anio"].astype(str) + "-" + df_ipc["mes"].astype(str) + "-01")


In [6]:
# Seleccionar solo el índice IPC General
ipc_values = df_ipc[["indice_ipc_general"]].values

In [7]:
# Normalizar los valores
scaler = MinMaxScaler(feature_range=(0, 1))
ipc_scaled = scaler.fit_transform(ipc_values)

In [8]:
# Función para crear secuencias de datos
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [9]:
# Crear secuencias con 12 meses de historial
sequence_length = 12
X, y = create_sequences(ipc_scaled, sequence_length)

In [10]:
# Dividir en entrenamiento (80%) y prueba (20%)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [11]:
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)


In [12]:
print(f"Tipo de X_train: {X_train.dtype}")


Tipo de X_train: float64


In [13]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)


In [14]:
print(f"Tipo de X_train: {X_train.dtype}")

Tipo de X_train: float32


In [ ]:
print(f"Tipo de X_train: {type(X_train)}")
print(f"Forma de X_train: {X_train.shape}")
print(f"Tipo de datos en X_train: {X_train.dtype}")

print(f"Tipo de y_train: {type(y_train)}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Tipo de datos en y_train: {y_train.dtype}")


Tipo de X_train: <class 'numpy.ndarray'>
Forma de X_train: (317, 12, 1)
Tipo de datos en X_train: float32
Tipo de y_train: <class 'numpy.ndarray'>
Forma de y_train: (317,)
Tipo de datos en y_train: float32


Tipo de X_train: <class 'numpy.ndarray'>
Forma de X_train: (317, 12, 1)
Tipo de datos en X_train: float32
Tipo de y_train: <class 'numpy.ndarray'>
Forma de y_train: (317,)
Tipo de datos en y_train: float32


In [ ]:
print("Hay NaN en X_train?", np.isnan(X_train).any())
print("Hay Inf en X_train?", np.isinf(X_train).any())


Hay NaN en X_train? False
Hay Inf en X_train? False


In [17]:
import tensorflow as tf
print("Versión de TensorFlow:", tf.__version__)


Versión de TensorFlow: 2.18.0


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(sequence_length, 1), dtype='float32'),
    Dropout(0.2),
    LSTM(100, return_sequences=False, dtype='float32'),
    Dropout(0.2),
    Dense(50, activation='relu', dtype='float32'),
    Dense(1, dtype='float32')
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 12, 100)        │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 126,301 (493.36 KB)

 Trainable params: 126,301 (493.36 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

In [36]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
16/20 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - loss: 3.5959e-04

20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 604ms/step - loss: 3.4467e-04 - val_loss: 0.0026
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.7300e-04 - val_loss: 0.0044
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.5300e-04 - val_loss: 0.0011
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - loss: 2.3955e-04 - val_loss: 0.0036
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 3.0712e-04 - val_loss: 0.0060
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 4.5429e-04 - val_loss: 0.0034
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 2.0371e-04 - val_loss: 0.0047
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.5066e-04 - val_loss: 0.0026
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.9654e-04 - val_loss: 9.3507e-04
Epoch 10/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 2.2749e-04 - val_loss: 5.2124e-04
Epoch 11/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 2.8265e-04 - val_loss: 0.0023
Epoch 12/50
20/20 ━━

In [ ]:
# Seleccionar los últimos 12 meses de datos como entrada
entrada = X_train[-1].reshape(1, sequence_length, 1)

predicciones_futuras = []

# Predecir mes a mes (3 meses)
for _ in range(3):
    prediccion = model.predict(entrada)  # Usar `entrada` en lugar de `X_test`
    predicciones_futuras.append(prediccion[0, 0])  # Extraer valor escalar

    # Actualizar la entrada eliminando el primer valor y agregando la nueva predicción
    entrada = np.append(entrada[:, 1:, :], [[[prediccion[0, 0]]]], axis=1)

# Convertir a array de NumPy
predicciones_futuras = np.array(predicciones_futuras).reshape(-1, 1)

# 🔹 **Desnormalizar las predicciones para obtener valores originales**
predicciones_reales = scaler.inverse_transform(predicciones_futuras)

print(predicciones_reales)  # Mostrará el IPC real predicho para los próximos 3 meses


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[[312.6374 ]
 [313.55222]
 [314.34317]]
